In [26]:
print("Loading modules, classes and methods...")
import multiprocessing
from unidecode import unidecode
from joblib import Parallel, delayed
import numpy as np
import string
import pandas as pd
import time
import os
import numpy as np
import re
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torchvision
import pickle
from unicodedata import normalize
from keras.preprocessing.text import Tokenizer
from torchvision import transforms, datasets
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt
from argparse import ArgumentParser
import textclassification as tc

Loading modules, classes and methods...


Using TensorFlow backend.


In [27]:
#GLOBAL VARIABLES
N_CORES = multiprocessing.cpu_count()

#Scores
STARTING         = 0
ENDING           = 2000000
BATCH_SCORES     = 10000

#Neural Network
TOP_WORDS       = 10000
REBUILD_DATA    = True
BIDIRECTIONAL   = True
EPOCHS          = 30
BATCH_SIZE      = 256

In [28]:
print("Loading data...")

valid = np.load("../light_data/new_classes.npy")
prevalences = pd.read_csv("../light_data/prevalencia_e_scores_com_siglas.csv")
dicionarioCsv = pd.read_csv("../light_data/DicionarioECG_completo.csv", index_col = 0)
siglasCsv = pd.read_csv("../light_data/DicionarioECG_Siglas.csv", index_col = 0)
db = pd.read_csv("../../data/DATA_LAUDOS_TEXTO_formato1", sep = ";")


print("Cutting slice. From {} to {}...".format(STARTING, ENDING))
db = db[STARTING:ENDING]
db = db.reset_index()

Loading data...
Cutting slice. From 0 to 2000000...


In [45]:
import pandas as pd

dicionarioCsv = pd.read_csv("../light_data/DicionarioECG_completo.csv", index_col = 0)
dicionario = {}
for row in dicionarioCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(" " + diag + " ")
    dicionario[row[0]] = aux

In [46]:
print("Preparing texts...")
texts = [" " + tc.clean_text(text) + " " for text in db["CONTEUDO"]]

siglas = {}
for row in siglasCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(" " + diag + " ")
    siglas[row[0]] = aux
siglas[11].append(" bav2:1 ")
    
dicionario = {}
for row in dicionarioCsv.itertuples():
    aux = []
    for diag in row:
        if type(diag) is str: aux.append(diag)
    dicionario[row[0]] = aux

Preparing texts...


In [205]:
print("Calculating scores...\n")

def return_scores(text, dicionario):
    scores = [max([tc.make_score(text, diag) for diag in dicionario[i]]) if valid[i][1] else 0 for i in range(74)]
    return scores

batch = BATCH_SCORES
print("Working in batches of", batch)
with open('../../data/resultados/'+ SCORES_FILE_NAME, 'w') as f:
    f.write("id_exame, scorings\n")
    #     errors = []

    
startTime = time.time()
for i in range(0, len(db), batch):
    print(i,"/",len(db))
    startBatch = time.time()
    #try:
#     scores = [return_scores(text, dicionario) for text in texts[i:i+batch]]
    scores = Parallel(n_jobs = N_CORES)(delayed(return_scores)
                              (text, dicionario)
                    for text in texts[i:i+batch])
    
    if(i == 0): result = np.array(scores)
    else: result = np.concatenate((totalScores, np.array(scores)), axis = 0)
    with open('../../data/resultados/' + SCORES_FILE_NAME, 'a') as f:
        for j in range(i,i+batch):
            f.write(str(db["ID_EXAME"][j]))
            f.write(',"')
            f.write(str(scores[j-i]))
            f.write('"\n')
#     except:
#         print("ERROR!!!!!")
#         errors.append([i, i+batch])
#     errors = np.array(errors)
#     np.save("errors.npy", errors)
    
    expectedTime = (((time.time() - startTime)/(i+batch)) * (len(db))) - (time.time() - startTime)
    timeBatch    = time.time() - startBatch
    print("This batch has been done in", int(timeBatch/60), "minutes and", timeBatch%60,"seconds!")
    print("Expected time for ending is around", int(expectedTime/3600), "hours and ", int((expectedTime%3600)/60),"minutes!")
#         with open('../../data/resultados/scorings1.csv', 'a') as f:
#             for j in range(i,i+batch):
#                 f.write(str(db["ID_EXAME"][j]))
#                 f.write(',"')
#                 f.write(str(scores[j-i]))
#                 f.write('"\n')
print("Y of training data defined!!! Saving...")
np.save("output/score_values.npy", result)
print("Saved!")

Calculating scores...

Working in batches of 100
0 / 1000000
This batch has been done in 0 minutes and 3.968424081802368 seconds!
Expected time for ending is around 11 hours and  1 minutes!
100 / 1000000
This batch has been done in 0 minutes and 3.566516399383545 seconds!
Expected time for ending is around 10 hours and  28 minutes!
200 / 1000000
This batch has been done in 0 minutes and 3.5585811138153076 seconds!
Expected time for ending is around 10 hours and  16 minutes!
300 / 1000000
This batch has been done in 0 minutes and 3.494030714035034 seconds!
Expected time for ending is around 10 hours and  7 minutes!
400 / 1000000
This batch has been done in 0 minutes and 3.689072370529175 seconds!
Expected time for ending is around 10 hours and  9 minutes!
500 / 1000000
This batch has been done in 0 minutes and 3.4898159503936768 seconds!
Expected time for ending is around 10 hours and  4 minutes!
600 / 1000000


KeyboardInterrupt: 

In [31]:
result = np.load("output/score_values.npy")

In [47]:
dicionario

{0: ['área eletricamente inativa',
  'zona eletricamente inativa',
  'infarto prévio',
  'IAM prévio',
  'fibrose',
  'inatividade elétrica',
  'zona inativa',
  'área inativa',
  'ondas q patológicas'],
 1: ['Bloqueio de ramo direito#', 'Bloqueio do ramo direito#', 'BRD!'],
 2: ['Bloqueio de ramo esquerdo#', 'Bloqueio do ramo esquerdo'],
 3: ['Bloqueio de ramo direito e bloqueio divisional anterossuperior do ramo esquerdo',
  'bloqueio de ramo direito e hemibloqueio anterior esquerdo',
  'bloqueio de ramo direito e bloqueio anterossuperior do ramo esquerdo',
  'BRD e BDAS',
  'BRD e BDASE',
  'BRD e HBAE'],
 4: ['Bloqueio intraventricular inespecífico', 'BIV?'],
 5: ['Sobrecarga ventricular esquerda (critérios de Romhilt-Estes)',
  'padrão strain',
  'Hipertrofia ventricular esquerda',
  'Sobrecarga ventricular esquerda'],
 6: ['Sobrecarga ventricular esquerda (critérios de voltagem)',
  'Sokolow Lyon'],
 7: ['Fibrilação atrial', 'Fibrilação auricular'],
 8: ['Flutter atrial', 'Flutte

In [ ]:
def return_scores(text, dicionario):
    scores = [max([tc.make_score(text, diag) for diag in dicionario[i]])]
    return scores

In [ ]:
print("Calculating scores...\n")

def return_scores(text, dicionario):
    scores = [max([tc.make_score(text, diag) for diag in dicionario[i]]) if valid[i][1] else 0 for i in range(74)]
    return scores

batch = BATCH_SCORES
print("Working in batches of", batch)
with open('../../data/resultados/'+ SCORES_FILE_NAME, 'w') as f:
    f.write("id_exame, scorings\n")
    #     errors = []

    
startTime = time.time()
for i in range(0, len(db), batch):
    print(i,"/",len(db))
    startBatch = time.time()
    #try:
#     scores = [return_scores(text, dicionario) for text in texts[i:i+batch]]
    scores = Parallel(n_jobs = N_CORES)(delayed(return_scores)
                              (text, dicionario)
                    for text in texts[i:i+batch])
    
    if(i == 0): result = np.array(scores)
    else: result = np.concatenate((totalScores, np.array(scores)), axis = 0)
    with open('../../data/resultados/' + SCORES_FILE_NAME, 'a') as f:
        for j in range(i,i+batch):
            f.write(str(db["ID_EXAME"][j]))
            f.write(',"')
            f.write(str(scores[j-i]))
            f.write('"\n')
#     except:
#         print("ERROR!!!!!")
#         errors.append([i, i+batch])
#     errors = np.array(errors)
#     np.save("errors.npy", errors)
    
    expectedTime = (((time.time() - startTime)/(i+batch)) * (len(db))) - (time.time() - startTime)
    timeBatch    = time.time() - startBatch
    print("This batch has been done in", int(timeBatch/60), "minutes and", timeBatch%60,"seconds!")
    print("Expected time for ending is around", int(expectedTime/3600), "hours and ", int((expectedTime%3600)/60),"minutes!")
#         with open('../../data/resultados/scorings1.csv', 'a') as f:
#             for j in range(i,i+batch):
#                 f.write(str(db["ID_EXAME"][j]))
#                 f.write(',"')
#                 f.write(str(scores[j-i]))
#                 f.write('"\n')
print("Y of training data defined!!! Saving...")
# np.save("output/score_values.npy", result)
print("Saved!")

In [284]:
print("Calculating thresholds and creating binary arrays...")
scores_thresh = []
y_bin = np.zeros((len(result),valid[:,1].sum()))
nClass = 0
for i in range(len(result[0])):
    if(not valid[i][1]): continue
    print("Calculating for", prevalences.loc[i]["Diagnostico"])
    temp_result       = result[:,i]
    temp_result = np.sort(temp_result, kind = 'mergesort')
    ocurrences     = len(result) * prevalences.loc[i]["Prevalencia"]
    ocurrences     = math.ceil(ocurrences)
    scores_thresh.append(temp_result[-ocurrences])
    for j in range(len(result)):
        if result[j][i] >= scores_thresh[-1]: y_bin[j][nClass] = 1
    print("Threshold =",temp_result[-ocurrences])
    nClass += 1

np.save("output/score_bin.npy", y_bin)
np.save("output/score_thresholds.npy", scores_thresh)

Calculating thresholds and creating binary arrays...
Calculating for área_eletricamente_inativa
Threshold = 88.889
Calculating for Bloqueio_de_ramo_direito
Threshold = 92.308
Calculating for Bloqueio_de_ramo_esquerdo
Threshold = 92.593
Calculating for Sobrecarga_ventricular_esquerda_(critérios_de_Romhilt-Estes)
Threshold = 93.939
Calculating for Fibrilação_atrial
Threshold = 78.947
Calculating for Flutter_atrial
Threshold = 93.75
Calculating for Bloqueio_atrioventricular_de_2°_grau_Mobitz_I
Threshold = 76.596
Calculating for Pré-excitação_ventricular_tipo_Wolff-Parkinson-White
Threshold = 77.778
Calculating for Sistema_de_estimulação_cardíaca_normofuncionante
Threshold = 62.162
Calculating for Taquicardia_atrial_multifocal
Threshold = 53.125
Calculating for Taquicardia_supraventricular
Threshold = 70.968
Calculating for Alterações_primárias_da_repolarização_ventricular
Threshold = 73.077
Calculating for Extrassístoles_supraventriculares
Threshold = 80.0
Calculating for Extrassístoles_v

In [297]:
print("Calculating the greenzone...")
gZoneIdx = []
gZoneIds = []
gZoneLen = 0
for i in range(len(result)):
    if y_bin[i].sum() > 0:
        gZoneIds.append(db["ID_EXAME"][i])
        gZoneIdx.append(i)
        gZoneLen += 1
print("Done! GreenZone has", gZoneLen, "registers!")

Calculating the greenzone...
Done! GreenZone has 537 registers!


In [305]:
print("Creating neural network...")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on a GPU :D")
else:
    device = torch.device("cpu")
    print("Running on a CPU :/")


class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class Net(nn.Module):
    
    def __init__(self, seq_size):
        
        print("Building NN...")
        embedding_dim = 128
        lstm_out_dim = 128
        num_embeddings = TOP_WORDS
        num_of_classes = 35
        
        super().__init__()
        #Camada de Embedding, o padding_idx é um argumento que eu descobri que é usada para falar para a camada que os números no fim de cada vetor são apenas lixo
        self.l1 = nn.Embedding(num_embeddings, embedding_dim, padding_idx = 0)
        #Eu não entendo muito bem o que essa camada faz. Pelo que eu entendi é algo probabilístico. Mas ela n altera o shape.
#         self.l2 = nn.Dropout(p=0.4)
        #A LSTM recebe os Embeddings e cospe o mesmo número de vetores que eu passei para ela. Não sei se eu deveria alterar o número de camadas da LSTM.
        #Se usar menos de 2 não dá pra colocar Dropout pq o Dropout é aplicado em todas as camadas menos na última.
        self.l3 = nn.LSTM(embedding_dim, lstm_out_dim, dropout = 0.2, num_layers = 2, bidirectional = BIDIRECTIONAL)
        #É o seguinte. Como as dimensões de entrada são estáticas, eu adicionei elas manualmente na camada linear para conseguir fazer o flatten.
        self.l4 = nn.Flatten()
        #Dimensao do vetor de entrada X dimensao da lstm
        self.l5 = nn.Linear(seq_size * lstm_out_dim * (2 if BIDIRECTIONAL else 1), num_of_classes)
        
    
    def forward(self, x):
        #Aqui eu só to passando o input pelas camadas mesmo
        x    = self.l1(x)
#         x    = self.l2(x)
        #A camada de LSTM retorna uma tupla, o vetor que eu quero é a primeira posição da tupla, por isso recebo assim.
        #Acho que a segunda camada da LSTM só é util ao passar de uma camada da LSTM para a outra.
        x, _ = self.l3(x)
        x    = self.l4(x)
        x    = self.l5(x)
        #Aqui eu aplico o softmax. Especifico o número de dimensões para ser um e tal. Não sei o que não está funcionando :c.
        x    = F.softmax(x, dim = 1)
            
        return x

Creating neural network...
Running on a GPU :D


In [347]:
print("Creating vocabulary...")
tokenizer = Tokenizer(num_words = TOP_WORDS, split = ' ')
tokenizer.fit_on_texts(texts)
print("Tokenizing texts...")
train_X   = tokenizer.texts_to_sequences(texts)
with open("output/tokenizer_"+str(TOP_WORDS)+".pickle", 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("X of training data defined!!! Saving...")
np.save("output/x_train.npy", train_X)

Creating vocabulary...


KeyboardInterrupt: 

In [324]:
print("Reshaping training data...")
train_X  = [torch.Tensor(i).type(torch.LongTensor) for i in train_X]
train_X  = pad_sequence(train_X, batch_first=True).type(torch.LongTensor)
train_y  = torch.Tensor(y_bin)
seq_size = len(train_X[0])
print("Training input has",seq_size,"dimensions!")

Reshaping training data...
Training input has 203 dimensions!


In [346]:
net = Net(seq_size).to(device)
optimizer = optim.Adam(net.parameters(), lr = 0.0005)
loss_function = nn.BCELoss()
# Training the model
print("\nAlexa, play eye of the tiger. It's training time!\n\n")
for epoch in range(EPOCHS):
    print("Epoch", epoch+1)
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
#         print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE]
        batch_y = train_y.squeeze()[i:i+BATCH_SIZE]

        batch_X, batch_y = batch_X.to(device), batch_y.to(device)

        net.zero_grad()
        outputs = net(batch_X)
        loss    = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print("Loss:", loss.data.tolist(),"\n")

print("Finished training!")
print("Saving model...")
torch.save(net.state_dict(), "output/network_model.pth")
print("\nCompleted!\nYou can find all the files that this script has generated in the output folder.")

  0%|          | 0/3 [00:00<?, ?it/s]

Building NN...

Alexa, play eye of the tiger. It's training time!


Epoch 1


 33%|███▎      | 1/3 [00:00<00:00,  9.44it/s]

Loss: 0.23120196163654327 

Epoch 2


 67%|██████▋   | 2/3 [00:00<00:00, 10.65it/s]

...

Loss: 0.12483090162277222 

Epoch 29


 67%|██████▋   | 2/3 [00:00<00:00, 10.69it/s]

Loss: 0.12229582667350769 

Epoch 30


100%|██████████| 3/3 [00:00<00:00, 12.94it/s]

Loss: 0.12250684946775436 

Finished training!
Saving model...

Completed!
You can find all the files that this script has generated in the output folder.
